### Q1.Design a Sudoku puzzle where the board consists of 81 squares, some of which are initially filled with digits from 1 to 9.  

The puzzle is to fill in all the remaining squares such that no digit appears twice in any row, column, or 3 × 3 box.  
A row, column, or box is called a unit.  

1.	Represent the Sudoku problem in  CSP by identifying the variable, domain, and constraint.
2.	Implement the problem using backtracking search, what is avg. time taken by the algorithm for 10 runs.
3.	Analyse how different fault finding algorithms such as Forward Checking, Arc consistency improve the computational time of backtracking search?
4.	Analyse how different Heuristics MRV (Minimum Remaining Values), Degree heuristic, Least Constraining Value affect the  computational time of backtracking search?


In [12]:
base  = 3
side  = base*base

# pattern for a baseline valid solution
def pattern(r,c): return (base*(r%base)+r//base+c)%side

# randomize rows, columns and numbers (of valid base pattern)
from random import sample
def shuffle(s): return sample(s,len(s)) 
rBase = range(base) 
rows  = [ g*base + r for g in shuffle(rBase) for r in shuffle(rBase) ] 
cols  = [ g*base + c for g in shuffle(rBase) for c in shuffle(rBase) ]
nums  = shuffle(range(1,base*base+1))

# produce board using randomized baseline pattern
solution = [ [nums[pattern(r,c)] for c in cols] for r in rows ]
print(solution)
board = np.array(solution)

# original_array = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# # Deep copy of the array
# copied_array = original_array.copy()

squares = side*side
empties = squares * 3//4
for p in sample(range(squares),empties):
    board[p//side][p%side] = 0

numSize = len(str(side))
for line in board:
    print(*(f"{n or '.':{numSize}} " for n in line))
print(solution)

[[8, 7, 2, 4, 6, 9, 3, 1, 5], [9, 6, 4, 5, 1, 3, 8, 7, 2], [3, 1, 5, 2, 7, 8, 9, 6, 4], [5, 3, 7, 6, 8, 2, 4, 9, 1], [2, 8, 6, 1, 9, 4, 5, 3, 7], [4, 9, 1, 7, 3, 5, 2, 8, 6], [6, 2, 9, 3, 4, 1, 7, 5, 8], [1, 4, 3, 8, 5, 7, 6, 2, 9], [7, 5, 8, 9, 2, 6, 1, 4, 3]]


NameError: name 'np' is not defined

In [2]:
def shortSudokuSolve(board):
    side   = len(board)
    base   = int(side**0.5)
    board  = [n for row in board for n in row ]
    blanks = [i for i,n in enumerate(board) if n==0 ]
    cover  = { (n,p):{*zip([2*side+r, side+c, r//base*base+c//base],[n]*(n and 3))}
                for p in range(side*side) for r,c in [divmod(p,side)] for n in range(side+1) }
    used   = set().union(*(cover[n,p] for p,n in enumerate(board) if n))
    placed = 0
    while placed>=0 and placed<len(blanks):
        pos        = blanks[placed]
        used      -= cover[board[pos],pos]
        board[pos] = next((n for n in range(board[pos]+1,side+1) if not cover[n,pos]&used),0)
        used      |= cover[board[pos],pos]
        placed    += 1 if board[pos] else -1
        if placed == len(blanks):
            solution = [board[r:r+side] for r in range(0,side*side,side)]
            yield solution
            placed -= 1

In [5]:
from IPython.display import display, HTML

def sudoku_to_html(board):
    """
    Generates an HTML table representation of a Sudoku board with thick edges for subgrids.

    Args:
        board (list of list of int): 2D list representing the Sudoku board.
                                     Empty cells should be represented by 0 or '.'.

    Returns:
        str: HTML string for the Sudoku board.
    """
    html = """
    <style>
        table { border-collapse: collapse; font-family: Arial, sans-serif; }
        td { border: 1px solid black; height: 40px; width: 40px; text-align: center; font-size: 18px; }
        .thick-border-top { border-top: 3px solid black; }
        .thick-border-left { border-left: 3px solid black; }
    </style>
    <table>
    """

    for i, row in enumerate(board):
        html += "<tr>"
        for j, cell in enumerate(row):
            # Apply thicker borders for subgrid boundaries
            classes = []
            if i % 3 == 0 and i != 0:
                classes.append("thick-border-top")
            if j % 3 == 0 and j != 0:
                classes.append("thick-border-left")
            
            class_attr = f'class="{" ".join(classes)}"' if classes else ""
            html += f"<td {class_attr}>{cell if cell != 0 else ''}</td>"
        html += "</tr>"

    html += "</table>"
    return html

In [7]:
# Generate and display the Sudoku board as HTML
html_output = sudoku_to_html(board)
display(HTML(html_output))

html_output = sudoku_to_html(solution)
display(HTML(html_output))


,5,8,,,2,7,6,
9,,4,,,,,,
,6,,3,,5,,,
,,,,2,,,,4
,,,,,,,,
,,,,,3,2,,
,,3,,9,,,4,
,,9,,1,4,,,
,,,,,,9,,


,5,8,,,2,7,6,
9,,4,,,,,,
,6,,3,,5,,,
,,,,2,,,,4
,,,,,,,,
,,,,,3,2,,
,,3,,9,,,4,
,,9,,1,4,,,
,,,,,,9,,


In [ ]:
display(HTML(html_output))

In [3]:
import random
from itertools import islice
while True:
    solved  = [*islice(shortSudokuSolve(board),2)]
    if len(solved)==1:break
    diffPos = [(r,c) for r in range(9) for c in range(9)
               if solved[0][r][c] != solved[1][r][c] ] 
    r,c = random.choice(diffPos)
    board[r][c] = solution[r][c]

KeyboardInterrupt: 